In [ ]:
import requests
import os

from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

In [ ]:
# Environment variables
load_dotenv()

# APIs
# geocoding_uri = 'https://nominatim.openstreetmap.org/search'
GEOCODE_URI = 'https://api.opencagedata.com/geocode/v1/json'
WEATHER_URI = 'https://api.tomorrow.io/v4/weather/forecast'
POI_URI = 'https://places-api.foursquare.com/places/search'

In [ ]:
# Initialize LLM and custom prompt
llm = init_chat_model('gemini-2.0-flash', model_provider='google_genai')
prompt = PromptTemplate.from_template("You are an assistant for giving rich descriptions on locations and cities around the world for tourism. Using your knowledge base, and given the following weather information and at most 3 points of interest chosen at your discretion in that location, give a rich description of the location. Be creative.\nLocation: {location}\nLocation information: {location_information}\nAnswer:")

In [ ]:
# State class for LangChain Graph
class State(TypedDict):
    location: str
    location_information: str
    answer: str

In [ ]:
# Retrieve information from APIs

def retrieve_location_info(state: State):
    # Retrieve latitude and longtitude from geocode api
    query = state['location']
    geo_response = requests.get(GEOCODE_URI, params={'q': query, 'key': os.environ['OPENCAGE_API_KEY']})
    lat = geo_response.json()['results'][0]['geometry']['lat']
    lon = geo_response.json()['results'][0]['geometry']['lng']

    # Retrieve weather information from coordinates
    weather_headers = {
    "accept": "application/json",
    "accept-encoding": "deflate, gzip, br"
    }
    weather_query = f"{lat},{lon}"
    weather_response = requests.get(WEATHER_URI, headers=weather_headers, params={'location': weather_query, 'apikey': os.environ['WEATHER_API_KEY']})
    
    weather_str = ['Weather Information for This Location for the Next 6 Days: \n\n']
    for day in weather_response.json()['timelines']['daily']:
        weather_str.append(f"{day['time'][:10]} | Max Temp: {day['values']['temperatureMax']}°C | Feels Like: {day['values']['temperatureApparentMax']}°C\n"
        f"Avg Humidity: {day['values']['humidityAvg']}%\n"
        f"Rain Probability: {day['values']['precipitationProbabilityMax']}%\n\n")
    weather_str = ''.join(weather_str)
        
    # Retrieve POI information from coordinates
    poi_headers = {
    "accept": "application/json",
    "X-Places-Api-Version": "2025-06-17",
    "authorization": f"Bearer {os.environ['POI_API_KEY']}"
    }
    category_ids = '4bf58dd8d48988d182941735,4bf58dd8d48988d181941735,4d4b7105d754a06377d81259'
    poi_response = requests.get(POI_URI, headers=poi_headers, params={'ll': weather_query, 'radius': 100000, 'fsq_category_ids': category_ids, 'fields': 'name,categories,location', 'limit': 20})

    poi_str = ['Points of Interest in This Location: \n\n']
    for location in poi_response.json()['results']:
        poi_str.append(f"POI Type: {location['categories'][0]['name']}\n"
        f"Name: {location['name']}\n"
        f"Address: {location['location']['formatted_address']}\n\n")
    poi_str = ''.join(poi_str)
    
    # Concatenate and return full information
    return {'location_information': ''.join([weather_str, poi_str])}

In [ ]:
# Generate LLM response
def generate(state: State):
    messages = prompt.invoke({'location': state['location'], 'location_information': state['location_information']})
    response = llm.invoke(messages)
    return {'answer': response.content}

In [ ]:
# Create graph workflow
graph_builder = StateGraph(State).add_sequence([retrieve_location_info, generate])
graph_builder.add_edge(START, 'retrieve_location_info')   
graph = graph_builder.compile() 

In [323]:
result = graph.invoke({'location': 'Munich, Germany'})
print(f"Answer: \n{result['answer']}")

Answer: 
Step into Munich in mid-July, where a delightful blend of Bavarian charm and summery warmth awaits. For the first half of your visit, expect temperatures dancing around the high 20s to low 30s Celsius, feeling just as pleasant with moderate humidity and negligible chance of rain. Perfect weather for strolling through the city's many parks and plazas. The latter half of the week brings slightly cooler temperatures, with a higher chance of rain, so pack an umbrella just in case!

Imagine yourself standing in the heart of the city at **Marienplatz**, the vibrant central square. The sun warms your face as you gaze upon the impressive Neues Rathaus (New Town Hall), its intricate facade teeming with gothic details. Listen for the melodic chimes of the Glockenspiel, a historical clock that reenacts stories from Munich's past with charming figurines. Street performers add to the lively atmosphere, creating a symphony of sights and sounds.

From there, escape the urban bustle with a le